# 📊📚 Analysez des données de systèmes éducatifs

## Sommaire
[Introduction](#Introduction)  

[1. Librairies et chargement des data](#1.-Librairies-et-chargement-des-data)
- [a. Librairies importées](#a.-Librairies-importées)
- [b. Chargement des fichiers CSV](#b.-Chargement-des-fichiers-CSV)

[2. Nettoyage des dataframes](#2.-Nettoyage-des-dataframes)
- [a. Création du log df](#a.-Création-du-log-df)
- [b. Suppression des colonnes avec na_rate = 1](#b.-Suppression-des-colonnes-avec-na_rate-=-1)
- [c. Suppression des faux pays](#c.-Suppression-des-faux-pays)

## Introduction

Academy est une plateforme d'éducation en ligne pour les étudiants du niveau lycée et université.
La start-up qui a développé la plateforme souhaite l'étendre à un niveau internationnal. 

Pour ce faire, il faut dans un premier temps **s'assurer que les données récoltées de la banque mondiale sont exploitables** pour enrichir la réflexion. 
Puis, dans un second temps, **identifier les pays cibles avec un fort potentiel** et voir leur évolution dans le temps.

### 1. Librairies et chargement des data

#### a. Librairies importées

In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

#### b. Chargement des fichiers CSV

In [3]:
df_data = pd.read_csv('../dataset/EdStatsData.csv')
df_country_series = pd.read_csv('../dataset/EdStatsCountry-Series.csv')
df_country = pd.read_csv('../dataset/EdStatsCountry.csv')
df_foot_note = pd.read_csv('../dataset/EdStatsFootNote.csv')
df_stats_series = pd.read_csv('../dataset/EdStatsSeries.csv')

###### Description des DF

**df_data** : Ensemble des indicateurs par pays et par années (1970 à 2100). Chaque ligne représente le couple pays/indicateurs.

**df_country** : Liste d'informations complémentaires concernant les pays. Chaque ligne correspond à un pays.

**df_country_series** : Indicateurs démographiques et économiques par année et par pays présents dans df_data. Chaque ligne repésente le couples pays/indicateurs/années.

**df_stats_series** : Lexique des indicateurs. Chaque lique correspond à un indicateur.

**df_foot_note** : Informations additionnelles sur les indicateurs basées sur l'année, le type d'indicateur et le pays dans df_data.

In [4]:
#add widget to display df.head(5)

### 2. Nettoyage des dataframes

#### a. Création du log df 

In [5]:
#Log df infos
dic = {"df_data" : df_data, "df_country":df_country, "df_country_series" : df_country_series, 
           "df_foot_note" : df_foot_note, "df_stats_series" : df_stats_series}


cat_log = ['shape','duplicated_rows','na_rate', 'stats','nunique_cat']

obj_log = {
    'log_cat': cat_log,
    "df_data": [None] * len(cat_log),
    "df_country": [None] * len(cat_log),
    "df_country_series": [None] * len(cat_log),
    "df_foot_note": [None] * len(cat_log),
    "df_stats_series": [None] * len(cat_log)
}

df_log = pd.DataFrame(obj_log)

In [6]:
dic_keys = list(dic.keys())

for df_name in dic_keys:
    
    #Shape
    shape_value = dic[df_name].shape
    df_log.loc[df_log['log_cat'] == 'shape', df_name] = [shape_value]
    
    #duplicated_rows
    duplicates = dic[df_name].duplicated().sum()
    df_log.loc[df_log['log_cat'] == 'duplicated_rows', df_name] = [duplicates]
    
    #na_rate
    na_rate = round(dic[df_name].isna().sum() / len(dic[df_name]),2)
    df_log.loc[df_log['log_cat'] == 'na_rate', df_name] = [na_rate]
    
    #stats df.describe(include=[np.number])
    stats_report = dic[df_name].describe(include=[np.number])
    df_log.loc[df_log['log_cat'] == 'stats', df_name] = [stats_report]
    
    #Nb uniques categories df_obj = df.select_dtypes(include='object')
    unique_categories = dic[df_name].select_dtypes(include='object').nunique()
    df_log.loc[df_log['log_cat'] == 'nunique_cat', df_name] = [unique_categories]

In [7]:
df_log

,log_cat,df_data,df_country,df_country_series,df_foot_note,df_stats_series
0,shape,"(886930, 70)","(241, 32)","(613, 4)","(643638, 5)","(3665, 21)"
1,duplicated_rows,0,0,0,0,0
2,na_rate,Country Name 0.00 Country Code 0.00 ...,Country Code ...,CountryCode 0.0 SeriesCode 0.0 DESCRIPT...,CountryCode 0.0 SeriesCode 0.0 Year ...,Series Code 0.00 To...
3,stats,1970 1971 197...,National accounts reference year Lates...,Unnamed: 3 count 0.0 mean ...,Unnamed: 4 count 0.0 mean ...,Unit of measure Notes from original so...
4,nunique_cat,Country Name 242 Country Code 242 ...,Country Code ...,CountryCode 211 SeriesCode 21 DESCRIPT...,CountryCode 239 SeriesCode 1558 Year ...,Series Code 3665 To...


In [8]:
# Widgets : Toggle pour log_cat et Toggle pour les colonnes
toggle_cat = widgets.ToggleButtons(
    options=df_log['log_cat'].values,
    description='Category:',
    button_style=''
)

toggle_df = widgets.ToggleButtons(
    options=df_log.columns[1:],
    description='Dataframe:',
    button_style=''
)

# Output widget pour afficher le résultat
out = widgets.Output()

# Callback qui combine les deux sélections
def update_value(change):
    category_name = toggle_cat.value
    df_name = toggle_df.value
    
    with out:
        out.clear_output()
        try:
            valeur = df_log.loc[df_log['log_cat'] == category_name, df_name].iloc[0]
            
            if isinstance(valeur, pd.Series):
                display(valeur.to_frame())
            else:
                display(valeur)
                
        except Exception as e:
            print("Erreur :", e)

# Observer les changements sur les deux widgets
toggle_cat.observe(update_value, names='value')
toggle_df.observe(update_value, names='value')

# Affichage
display(toggle_cat, toggle_df, out)

ToggleButtons(description='Category:', options=('shape', 'duplicated_rows', 'na_rate', 'stats', 'nunique_cat')…

ToggleButtons(description='Dataframe:', options=('df_data', 'df_country', 'df_country_series', 'df_foot_note',…

Output()

#### b. Suppression des colonnes avec na_rate = 1

In [10]:
if df_data.loc[:,['Country Name','Indicator Name']].duplicated().sum() == 0:
    print('✅ Pas de doublon pour le couple Pays/Indicateurs dans le df_data')

✅ Pas de doublon pour le couple Pays/Indicateurs dans le df_data


In [11]:
col_to_delete = []

for df_name in dic_keys:
    temp_df = df_log.loc[df_log['log_cat'] == "na_rate", df_name].iloc[0].reset_index()
    
    #col name
    new_col_name = ['col_name','na_rate']
    temp_df.columns = new_col_name
    temp_df_filtered = temp_df[temp_df['na_rate'] == 1]
    col_list = temp_df_filtered['col_name'].astype(str).tolist()
    
    for col in col_list:
        col_to_delete.append(col)

print(col_to_delete)

['2017', 'Unnamed: 69', 'Unnamed: 31', 'Unnamed: 3', 'Unnamed: 4', 'Unit of measure', 'Limitations and exceptions', 'Notes from original source', 'General comments', 'Development relevance', 'Other web links', 'Related indicators', 'License Type', 'Unnamed: 20']


In [12]:
col_to_delete_set = set(col_to_delete)

for df_name in dic_keys:
    col_to_keep = [x for x in dic[df_name].columns if x not in col_to_delete_set]
    dic[df_name] = dic[df_name][col_to_keep]

### c. Suppression des faux pays

In [13]:
#from df_country['Short Name'].unique()
region_to_delete = ['Arab World','East Asia & Pacific (developing only)',
       'East Asia & Pacific (all income levels)',
       'Europe & Central Asia (developing only)',
       'Europe & Central Asia (all income levels)','Euro area','European Union','High income',
       'Heavily indebted poor countries (HIPC)','Latin America & Caribbean (developing only)',
       'Least developed countries: UN classification', 'Low income','Lower middle income',
       'Low & middle income','Middle East & North Africa (all income levels)','Middle income',
       'OECD members','Sub-Saharan Africa (developing only)','Sub-Saharan Africa (all income levels)',
       'World']

In [14]:
df_region = pd.DataFrame({'Short Name' : region_to_delete})
df_region = pd.merge(df_region,df_country.loc[:,['Short Name','Country Code']], how='left', on='Short Name')

In [15]:
dic['df_country_series'] = dic['df_country_series'].rename(columns={'CountryCode' : 'Country Code'})
dic['df_foot_note'] = dic['df_foot_note'].rename(columns={'CountryCode' : 'Country Code'})

In [16]:
for df_name in dic_keys:
    if df_name != 'df_stats_series':
        dic[df_name] = dic[df_name][~dic[df_name]['Country Code'].isin(df_region['Country Code'].unique())]

In [9]:
dic['df_stats_series']['Topic'].unique()

array(['Attainment', 'Education Equality',
       'Infrastructure: Communications', 'Learning Outcomes',
       'Economic Policy & Debt: National accounts: US$ at current prices: Aggregate indicators',
       'Economic Policy & Debt: National accounts: US$ at constant 2010 prices: Aggregate indicators',
       'Economic Policy & Debt: Purchasing power parity',
       'Economic Policy & Debt: National accounts: Atlas GNI & GNI per capita',
       'Teachers', 'Education Management Information Systems (SABER)',
       'Early Child Development (SABER)',
       'Engaging the Private Sector (SABER)',
       'School Health and School Feeding (SABER)',
       'School Autonomy and Accountability (SABER)',
       'School Finance (SABER)', 'Student Assessment (SABER)',
       'Teachers (SABER)', 'Tertiary Education (SABER)',
       'Workforce Development (SABER)', 'Literacy', 'Background',
       'Primary', 'Secondary', 'Tertiary', 'Early Childhood Education',
       'Pre-Primary', 'Expenditures'

In [80]:
df_stats_series[df_stats_series['Topic'] == 'Attainment']['Indicator Name'].unique()

array(['Barro-Lee: Percentage of female population age 15-19 with no education',
       'Barro-Lee: Percentage of population age 15-19 with no education',
       'Barro-Lee: Percentage of female population age 15+ with no education',
       'Barro-Lee: Percentage of population age 15+ with no education',
       'Barro-Lee: Percentage of female population age 20-24 with no education',
       'Barro-Lee: Percentage of population age 20-24 with no education',
       'Barro-Lee: Percentage of female population age 25-29 with no education',
       'Barro-Lee: Percentage of population age 25-29 with no education',
       'Barro-Lee: Percentage of female population age 25+ with no education',
       'Barro-Lee: Percentage of population age 25+ with no education',
       'Barro-Lee: Percentage of female population age 30-34 with no education',
       'Barro-Lee: Percentage of population age 30-34 with no education',
       'Barro-Lee: Percentage of female population age 35-39 with no education

#### Critères de recherches. il faudrait que les pays :
 - aient une accessibilité restreinte du niveau secondaire et universitaire. Soit par un manque d'infrasturcture, soit par manque de personnel, soit réservé à une certaine population (classe sociale, genre...). Car l'objectif étant justement de pouvoir proposer une alternative.
 - aient une population jeune car in fine ce sont eux les clients finaux.
 - soient politiquement stables car l'éducation n'est peut-être pas une priorité en période de crise et administrativement cela peut comporter un risque pour la start-up. 
 - soient un minimum développé de sorte à ce que les étudiants puissent se procurer le matériel informatique adéquat.

In [84]:
dic_kpis = {
    'School accessibility': {
        'topic': {
            'Teachers': [
                'Teachers in secondary education, both sexes (number)',
                'Teachers in tertiary education programmes, both sexes (number)',
            ],
            'Education Equality': [
                'DHS: Secondary completion rate',
                'UIS: Total net attendance rate, lower secondary, both sexes (%)',
                'MICS: Secondary completion rate'
            ],
            'Attainment' : [
                'Barro-Lee: Percentage of population age 15+ with no education'
            ]
        }
    },
    'Demography': {
        'topic': {
            'Population': [
                'Population of the official age for secondary education, both sexes (number)'
            ],
            'Health: Population: Structure': [
                'Population, total'
            ],
            'Health: Population: Dynamics': [
                'Population growth (annual %)'
            ]
        }
    },
    'Political' : {
        'topic': {
            'xxx': [
                
            ]
        }
    },
    'Economic': {
        'topic' : {
            'Infrastructure: Communications' : [
                'Personal computers (per 100 people)',
                'Internet users (per 100 people)'
            ]
        }
    }
}


In [85]:
dic_kpis['Economic']['topic']

{'Infrastructure: Communications': ['Personal computers (per 100 people)',
  'Internet users (per 100 people)']}